In [7]:
import torch
from torch import nn
from torch.nn import Parameter
from torch.autograd import Variable
import numpy as np

la = 9
lb = 12
k = 2
macierz = torch.FloatTensor([
    [-5.8, -3.1, 4.5, 6.7, 14.3, 18.2, 20.1, 20.6, 15.9, 11.2, 3.6, -7.2],
    [-2.9, 3.6, 4.2, 9.7, 16.1, 19.5, 20.0, 18.8, 16.4, 7.6, 3.2, 1.3],
    [0.3, 1.5, 5.9, 8.4, 14.8, 20.2, 24.5, 24.7, 19.7, 13.0, 7.9, 1.9],
    [2.3, 6.5, 8.7, 9.2, 12.3, 15.4, 17.3, 20.0, 14.8, 10.8, 8.7, 6.4],
    [11.5, 13.9, 14.3, 15.7, 16.3, 17.4, 17.2, 17.7, 18.2, 17.4, 14.6, 10.4],
    [9.7, 10.3, 12.7, 15.5, 21.2, 22.1, 24.1, 25.3, 23.5, 20.1, 15.7, 11.8],
    [14.0, 15.6, 17.5, 20.3, 20.6, 18.1, 17.6, 18.2, 17.8, 16.8, 14.9, 16.0],
    [23.1, 23.3, 21.4, 19.0, 17.1, 15.5, 15.4, 15.6, 15.4, 18.6, 20.9, 21.3],
    [23.8, 24.6, 23.4, 20.8, 18.1, 15.1, 14.4, 14.5, 17.3, 19.0, 21.8, 24.3]
])

ustalone_a = torch.Tensor(la, k)
ustalone_b = torch.Tensor(lb, k)
pred_macierz = torch.zeros(la, lb, dtype=torch.float)

In [8]:
class Factorize(nn.Module):
	def __init__(self):
        	super(Factorize, self).__init__()
        	self.a_vector = Parameter(torch.Tensor(la, 1))
        	self.b_vector = Parameter(torch.Tensor(lb, 1))
        def forward(self):
        	output = torch.mm(self.a_vector, self.b_vector.transpose(0,1))+pred_macierz
		return output 


In [9]:
model = Factorize()
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
list(model.parameters())
desired_output = Variable(macierz)
model_file = open("model.txt", "w")
model_file.close()


In [10]:
for i in range(k):
	torch.save(model.state_dict(), "model.txt")
	for j in range(3):
		optimizer.zero_grad()
		output = model()
		loss = loss_func(output, desired_output)
		print loss, i, j
		loss.backward()
		optimizer.step()
	pred_macierz = pred_macierz+torch.mm(model.a_vector, model.b_vector.transpose(0,1))
	ustalone_a = model.a_vector
	ustalone_b = model.b_vector
	model = Factorize()
	model.load_state_dict(torch.load("model.txt"))

tensor(258.9745) 0 0
tensor(258.9745) 0 1
tensor(258.9745) 0 2
tensor(258.9745) 1 0
tensor(258.9745) 1 1


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.